In [7]:
import tensorflow as tf
from inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
import inception_preprocessing
from tensorflow.contrib.slim import arg_scope
from matplotlib import pyplot as plt
from matplotlib.image import imread
from skimage.transform import resize
from tqdm import tqdm_notebook as tqdm
import numpy as np
import os
%matplotlib inline
numClasses = 36;
def parseImages(filename, label=None):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_image(image_string, channels=3)
    image_resized = tf.image.resize_image_with_crop_or_pad(image_decoded, 299, 299)
    image_resized = tf.reshape(image_resized, [299,299,3])
    image_resized = tf.to_float(image_resized)
    return image_resized, label

def listImages(path):
    filenames = []
    labels = []
    for file in os.listdir(path):
        num = int(file[:3]) if int(file[:3]) < 36 else int(file[:3]) - 26
        num = num - 1
        #num = 1 if "dog" in file else 0
        labels.append(num)
        filenames.append(os.path.join(path,file))
    return filenames,labels

def trainingPreprocess(image, label):
    image = inception_preprocessing.preprocess_image(image, 299, 299, 1)
    return image, label

def testPreprocess(image, label):
    image = inception_preprocessing.preprocess_image(image, 299, 299, 1)
    return image, label


In [8]:
trainDir = r'C:\Users\drewm\Documents\Tensorflow Resources\IEEE CV\Datasets\Characters'
testDir = r'C:\Users\drewm\Documents\Tensorflow Resources\IEEE CV\Datasets\Characters'
#trainDir = r'C:\Users\drewm\Downloads\train\train'
#testDir = trainDir
model_path = r'C:\Users\drewm\Documents\Tensorflow Resources\IEEE CV\inception_resnet_v2.ckpt'

In [9]:
##Reset Graph##
tf.reset_default_graph()

##Create Dataset Objects##
sess = tf.Session()
filenames,labels = listImages(trainDir)
numFiles = len(filenames)
slices = [0.9, 0.1] #Train and Validation slices
filenames = tf.convert_to_tensor(filenames)
labels = tf.convert_to_tensor(labels)
##Train Dataset##
trainDataset = tf.data.Dataset.from_tensor_slices((filenames,labels))
trainDataset = trainDataset.shuffle(buffer_size=int(numFiles * 0.9))
trainDataset = trainDataset.map(parseImages)
trainDataset = trainDataset.map(trainingPreprocess)
##Validation Dataset##
validDataset = tf.data.Dataset.from_tensor_slices((filenames,labels)) #Use some for validation
validDataset = validDataset.shuffle(buffer_size=int(numFiles * 0.1))
validDataset = validDataset.map(parseImages)
validDataset = validDataset.map(trainingPreprocess)
##Test Dataset##
filenames,labels = listImages(testDir)
filenames = tf.convert_to_tensor(filenames)
testDataset = tf.data.Dataset.from_tensor_slices((filenames,labels))
testDataset = testDataset.shuffle(buffer_size=numFiles)
testDataset = testDataset.map(parseImages)
testDataset = testDataset.map(trainingPreprocess)

##Set up Iterator Object##
batch_size = tf.placeholder(tf.int64)
batchTrain = trainDataset.batch(batch_size)
print(batchTrain.output_shapes)
iterator = tf.data.Iterator.from_structure(batchTrain.output_types,batchTrain.output_shapes)
images, labels = iterator.get_next()
train_init_op = iterator.make_initializer(batchTrain)
valid_init_op = iterator.make_initializer(validDataset.batch(batch_size))
test_init_op = iterator.make_initializer(testDataset.batch(batch_size))

##Set up architecture of ResnetV2##
is_training = tf.placeholder(tf.bool)
dropout_keep_prob = tf.placeholder(tf.float32)
print(images.shape)

#NumClasses: all alphanumerics
with arg_scope(inception_resnet_v2_arg_scope()):
        logits, _ = inception_resnet_v2(images, num_classes=numClasses, is_training=is_training, dropout_keep_prob=dropout_keep_prob)
exclude = ['InceptionResnetV2/Logits', 'InceptionResnetV2/AuxLogits']
variables_to_restore = tf.contrib.framework.get_variables_to_restore(exclude=exclude)
init_fn = tf.contrib.framework.assign_from_checkpoint_fn(model_path, variables_to_restore)

##Set Up Optimizer and Restore Checkpoint##
loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
tf.summary.scalar('Loss', tf.reduce_mean(loss))
prediction = tf.to_int32(tf.argmax(logits, 1))
correct_prediction = tf.equal(prediction, labels)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('Training_Accuracy', accuracy)
summ = tf.summary.merge_all()

logits = tf.contrib.framework.get_variables('InceptionResnetV2/Logits')
logitsOptimizer = tf.train.AdamOptimizer(0.001)
logits_train_op = logitsOptimizer.minimize(loss, var_list=logits)
logitsInit = tf.variables_initializer(logits)
'''
auxLogits = tf.contrib.framework.get_variables('InceptionResnetV2/AuxLogits')
print(auxLogits)
auxLogitsOptimizer = tf.train.AdamOptimizer(0.001)
auxLogits_train_op = auxLogitsOptimizer.minimize(loss, var_list=auxLogits)
auxLogitsInit = tf.variables_initializer(auxLogits)
'''
optimizer = tf.train.AdamOptimizer(0.001)
total_train_op = optimizer.minimize(loss)

(TensorShape([Dimension(None), Dimension(299), Dimension(299), Dimension(3)]), TensorShape([Dimension(None)]))
(?, 299, 299, 3)


In [10]:
#sess = tf.InteractiveSession()
init_fn(sess)
#vars = tf.contrib.framework.get_variables('InceptionResnetV2/Logits')
#logitsInit = tf.initializers.variables(vars)
sess.run(logitsInit)
sess.run(tf.variables_initializer(logitsOptimizer.variables()))
#sess.run(auxLogitsInit)
#sess.run(tf.variables_initializer(auxLogitsOptimizer.variables()))
sess.run(tf.variables_initializer(optimizer.variables()))
sess.run(tf.global_variables_initializer())
writer = tf.summary.FileWriter(r'C:/Tensorboard/IEEE CV', graph=tf.get_default_graph())
saver = tf.train.Saver()

INFO:tensorflow:Restoring parameters from C:\Users\drewm\Documents\Tensorflow Resources\IEEE CV\inception_resnet_v2.ckpt


In [12]:
#FINE TUNING LAYER

epochs = 20
batchSize = 32
sumCVAcc = []

print('Batch Progress')
for epoch in tqdm(range(epochs)):
    CVAcc = []
    print('Starting epoch ' + str(epoch + 1) + ' / 20')
    sess.run(train_init_op, feed_dict={batch_size:batchSize})
    for batch in tqdm(range(int(numFiles * slices[0] / batchSize))):
        s, _ = sess.run((summ, logits_train_op), feed_dict={is_training:True, dropout_keep_prob:0.8})
        writer.add_summary(s, epoch * numFiles / batchSize + batch)
    sess.run(valid_init_op, feed_dict={batch_size:batchSize})
    saver.save(sess, r'C:\Users\drewm\Documents\Tensorflow Resources\IEEE CV\models\model' + str(epoch) + '\model', epoch)
    for i in range(int(numFiles * 0.1 / batchSize)):
        acc = sess.run(accuracy, feed_dict={is_training: False,dropout_keep_prob:1.0})
        CVAcc.append(acc)
        
    print("CV Accuracy: " + str(np.mean(CVAcc)))
    sumCVAcc.append(np.mean(CVAcc))

Batch Progress


HBox(children=(IntProgress(value=0, max=20), HTML(value='')))

Starting epoch 1 / 20


HBox(children=(IntProgress(value=0, max=216), HTML(value='')))

Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\drewm\Miniconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\drewm\Miniconda3\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\drewm\Miniconda3\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



...

CV Accuracy: 0.0
Starting epoch 10 / 20


HBox(children=(IntProgress(value=0, max=216), HTML(value='')))

CV Accuracy: 0.0
Starting epoch 11 / 20


HBox(children=(IntProgress(value=0, max=216), HTML(value='')))

KeyboardInterrupt: 

In [26]:
#FULL TRAIN CYCLE

batchSize = 4
print('Batch progress')
for epoch in tqdm(range(epochs)):
    CVAcc = []
    print('Starting full train epoch ' + str(epoch + 1) + ' / 20')
    sess.run(train_init_op, feed_dict={batch_size: batchSize})
    for batch in tqdm(range(int(numFiles * slices[0] / batchSize))):
        s, _ = sess.run((summ, total_train_op), feed_dict={is_training:True, dropout_keep_prob:0.8})
        writer.add_summary(s, epoch * numFiles / batchSize + batch)
    sess.run(valid_init_op, feed_dict={batch_size:batchSize})
    for i in range(int(numFiles * slices[1] / batchSize)):
        acc = sess.run(accuracy, feed_dict={is_training: False,dropout_keep_prob:1.0})
        CVAcc.append(acc)
    print("CV Accuracy: " + str(np.mean(CVAcc)))
    sumCVAcc.append(np.mean(CVAcc))
    saver.save(sess, r'C:\Users\drewm\Documents\Tensorflow Resources\IEEE CV\models\model' + str(epoch + 10), epoch + 10)

Batch progress


HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

Starting full train epoch 1 / 20


HBox(children=(IntProgress(value=0, max=1733), HTML(value='')))

CV Accuracy: 0.0052083335
Starting full train epoch 2 / 20


...

CV Accuracy: 0.375
Starting full train epoch 9 / 20


HBox(children=(IntProgress(value=0, max=1733), HTML(value='')))

CV Accuracy: 0.23697917
Starting full train epoch 10 / 20


HBox(children=(IntProgress(value=0, max=1733), HTML(value='')))

CV Accuracy: 0.0026041667



In [11]:
f, l = sess.run((filenames,labels), feed_dict={is_training: False,dropout_keep_prob:1.0})

FailedPreconditionError: GetNext() failed because the iterator has not been initialized. Ensure that you have run the initializer operation for this iterator before getting the next element.
	 [[{{node IteratorGetNext}} = IteratorGetNext[output_shapes=[[?,299,299,3], [?]], output_types=[DT_FLOAT, DT_INT32], _device="/job:localhost/replica:0/task:0/device:CPU:0"](IteratorV2)]]

Caused by op 'IteratorGetNext', defined at:
  File "C:\Users\drewm\Miniconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\drewm\Miniconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\drewm\Miniconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\drewm\Miniconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\drewm\Miniconda3\lib\site-packages\ipykernel\kernelapp.py", line 478, in start
    self.io_loo